In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings,CacheBackedEmbeddings
from langchain.document_loaders import TextLoader
from langchain.schema.runnable import RunnablePassthrough,RunnableLambda
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.storage import LocalFileStore # 캐시 파일 저장 경로 지정용
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

CHAT = ChatOpenAI(
    temperature=0.5,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model = "gpt-4o-mini",
)

In [60]:
"""
강의와 달리 캐시 부분은 제외했습니다.
"""

### 문서를 로드한다
row_txt = TextLoader("../files/for_assignment.txt")

### 문서를 쪼갠다
# 문서를 쪼개줄 splitter
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
# splitter를 이용해서 문서 쪼개기
docs = row_txt.load_and_split(text_splitter=splitter)
#임베딩 객체
embeddings = OpenAIEmbeddings()
# 벡터 스토어 생성
vectorstore = FAISS.from_documents(docs, embeddings)

### 쪼개진 문서를 전달한다.
#메모리 객체
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="history"
)
# 대화내용을 기록 및 전달하기 위한 탬플릿
template = [
        (
            "system",
            """
            아래의 내용은 당신과 사용자가 이전에 나눴던 대화 내용입니다. 이후 답변에 참고하세요.
            /n/n
            {history}
            """,
        ),
        ("human", "이번 질문은'{question}'이야. 너는 질문의 내용과 이전 대화 내용이 존재할 경우 해당 대화 내용을 짧게 전달해주면 돼"),
    ]
# memoery_chain 생성. 이후에 메인 chain과 연결 할 예정. LLMChain 사용이 메모리 사용이 더 편해서 해당 방식 사용
memoery_chain = LLMChain(
    llm=CHAT,
    memory=memory,
    prompt=ChatPromptTemplate.from_messages(template)
)
# 문서를 전달할때 사용할 retriver 생성
retriver = vectorstore.as_retriever()
# 문서를 한 번에 받아서, 처리해줄 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            당신은 도움이 되는 조수입니다.
            다음 컨텍스트만 사용하여 질문에 답합니다.
            답을 모르면 모르겠다고만 하면 지어내지 마세요.:
            \n\n
            {context}
            \n\n
            {history}
            """,
        ),
        ("human", "{question}"),
    ]
)
# 체인 구성
chain = {"history":memoery_chain,"context":retriver,"question":RunnablePassthrough()} | prompt | CHAT


In [61]:
chain.invoke("Aaronson 은 유죄인가요?")

현재까지의 대화 내용은 없습니다. "Aaronson 은 유죄인가요?"라는 질문에 대해 답변할 수 있도록 추가 정보를 제공해 주시면 감사하겠습니다.Aaronson은 유죄가 아닙니다. 그에 대한 사진이 존재했으며, 그것은 그의 무죄를 증명하는 것이었습니다. 그러나 Winston은 그 사진을 본 적이 없으며, 그것은 존재하지 않았다고 스스로 믿게 되었습니다.

AIMessageChunk(content='Aaronson은 유죄가 아닙니다. 그에 대한 사진이 존재했으며, 그것은 그의 무죄를 증명하는 것이었습니다. 그러나 Winston은 그 사진을 본 적이 없으며, 그것은 존재하지 않았다고 스스로 믿게 되었습니다.')

In [62]:
chain.invoke("그(Aaronson)가 테이블에 어떤 메시지를 썼나요?")

이전 대화에서는 "Aaronson 은 유죄인가요?"라는 질문이 있었지만, 그에 대한 구체적인 내용이나 메시지는 언급되지 않았습니다. 현재로서는 Aaronson이 테이블에 쓴 메시지에 대한 정보가 없습니다. 추가 정보를 제공해 주시면 더 도움을 드릴 수 있습니다.현재로서는 Aaronson이 테이블에 쓴 메시지에 대한 정보가 없습니다. 추가 정보를 제공해 주시면 더 도움을 드릴 수 있습니다.

AIMessageChunk(content='현재로서는 Aaronson이 테이블에 쓴 메시지에 대한 정보가 없습니다. 추가 정보를 제공해 주시면 더 도움을 드릴 수 있습니다.')

In [63]:
chain.invoke("그가 테이블에 어떤 메시지를 썼나요?")

이전 대화에서 "Aaronson 은 유죄인가요?"라는 질문이 있었고, 그에 대한 구체적인 내용이나 메시지는 언급되지 않았습니다. 현재로서는 Aaronson이 테이블에 쓴 메시지에 대한 정보가 없습니다. 추가 정보를 제공해 주시면 더 도움을 드릴 수 있습니다.현재로서는 그가 테이블에 쓴 메시지에 대한 정보가 없습니다. 추가 정보를 제공해 주시면 더 도움을 드릴 수 있습니다.

AIMessageChunk(content='현재로서는 그가 테이블에 쓴 메시지에 대한 정보가 없습니다. 추가 정보를 제공해 주시면 더 도움을 드릴 수 있습니다.')

In [64]:
chain.invoke("Julia 는 누구인가요?")

이전 대화에서는 "Aaronson 은 유죄인가요?"라는 질문과 관련된 내용이 있었습니다. 그러나 "Julia"에 대한 언급이나 정보는 없었습니다. Julia에 대해 궁금한 점이나 추가 정보를 제공해 주시면 더 도움을 드릴 수 있습니다.현재 제공된 정보에서는 Julia에 대한 구체적인 설명이나 배경이 없습니다. Julia에 대해 더 알고 싶으시다면 추가 정보를 제공해 주시면 더 도움을 드릴 수 있습니다.

AIMessageChunk(content='현재 제공된 정보에서는 Julia에 대한 구체적인 설명이나 배경이 없습니다. Julia에 대해 더 알고 싶으시다면 추가 정보를 제공해 주시면 더 도움을 드릴 수 있습니다.')